<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/CodeFormer_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **セットアップ**

# Clone CodeFormer and enter the CodeFormer folder
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/cedro3/CodeFormer.git
%cd CodeFormer

# Set up the environment
# Install python dependencies
!pip install -r requirements.txt
# Install basicsr
!python basicsr/setup.py develop

# Download the pre-trained model
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

# Visualization function
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display result
from IPython.display import clear_output
import os
import glob
def display_result(input_folder, result_folder):
  input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
  for input_path in input_list:
    img_input = imread(input_path)
    basename = os.path.splitext(os.path.basename(input_path))[0]
    output_path = os.path.join(result_folder, basename+'.png')
    img_output = imread(output_path) 
    display(img_input, img_output)

# reset_folder
import shutil
def reset_folder(path):
    if os.path.isdir(path):
      shutil.rmtree(path)
    os.makedirs(path,exist_ok=True)


In [ ]:
#@title **クロップした顔のデモ**
input_folder = 'inputs/cropped_faces' 
w = 0.7 #@param {type:"slider", min:0.1, max:0.9, step:0.1}

! python inference_codeformer.py --w $w\
                                  --test_path $input_folder\
                                  --has_aligned

clear_output()
result_folder = 'results/cropped_faces_'+str(w)+'/restored_faces'
display_result(input_folder, result_folder )

In [ ]:
#@title **画像全体のデモ**
input_folder = 'inputs/whole_imgs' 
w = 0.7 #@param {type:"slider", min:0.1, max:0.9, step:0.1}


! python inference_codeformer.py --w $w\
                                  --test_path $input_folder\
                                  --bg_upsampler realesrgan

clear_output()
result_folder = 'results/whole_imgs_'+str(w)+'/final_results'
display_result(input_folder, result_folder )

In [ ]:
#@title **画像全体オリジナル**
import os
import shutil
from PIL import Image

pic ='01.jpg'#@param {type:"string"} 
input_folder = 'inputs/whole_imgs_original'
reset_folder(input_folder)
im = Image.open('pic/'+pic)
im.save(input_folder+'/'+os.path.splitext(pic)[0]+'.png')

w = 0.2 #@param {type:"slider", min:0.1, max:0.9, step:0.1}


! python inference_codeformer.py --w $w\
                                  --test_path $input_folder\
                                  --bg_upsampler realesrgan\
                                  --face_upsample

clear_output()
result_folder = 'results/whole_imgs_original_'+str(w)+'/final_results'
display_result(input_folder, result_folder )

In [ ]:
#@title **ダウンロード**
#@markdown -google chrome 専用
from google.colab import files
files.download(result_folder+'/'+os.path.splitext(pic)[0]+'.png')